# PostgreSQL Checkpointer 테스트

LangGraph의 PostgreSQL checkpointer를 사용해서 대화 이력 저장/복원을 테스트합니다.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# .env 파일 로드
dotenv_path = find_dotenv()
load_dotenv(dotenv_path, override=True)

import asyncio
import uuid
from app.agents.workflows.chat_workflow import process_chat
from app.infra.checkpointer import setup_checkpointer_tables

/Users/kimjunghyeon/Desktop/workspace/ai-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 체크포인터 테이블 설정

In [2]:
# 체크포인터 테이블 초기화
await setup_checkpointer_tables()
print("Checkpointer tables setup completed")

Checkpointer tables setup completed


## 2. 세션 생성 및 첫 번째 대화

In [3]:
# 새로운 세션 ID 생성
session_id = str(uuid.uuid4())
user_id = "test_user"

print(f"Session ID: {session_id}")

# 첫 번째 메시지
result1 = await process_chat(
    message="안녕하세요. 저는 김철수입니다.",
    user_id=user_id,
    session_id=session_id
)

print("첫 번째 응답:")
print(f"Answer: {result1['answer']}")
print(f"Question Type: {result1['question_type']}")

Session ID: c5da5b70-041e-4252-8f24-717c474e3c82
첫 번째 응답:
Answer: 안녕하세요, 김철수님! 만나서 반갑습니다. 어떻게 도와드릴까요?
Question Type: FACT


## 3. 같은 세션에서 두 번째 대화 (이력 유지 확인)

In [ ]:
# 두 번째 메시지 - 이전 대화 이력을 기억하는지 확인
result2 = await process_chat(
    message="제 이름이 뭐라고 했죠?",
    user_id=user_id,
    session_id=session_id  # 같은 세션 ID 사용
)

print("두 번째 응답:")
print(f"Answer: {result2['answer']}")
print(f"Question Type: {result2['question_type']}")

## 4. 새로운 세션에서 대화 (이력 분리 확인)

In [5]:
# 새로운 세션 ID로 대화
new_session_id = str(uuid.uuid4())
print(f"New Session ID: {new_session_id}")

result3 = await process_chat(
    message="제 이름이 뭐라고 했죠?",  # 같은 질문
    user_id=user_id,
    session_id=new_session_id  # 다른 세션 ID
)

print("새 세션 응답:")
print(f"Answer: {result3['answer']}")
print(f"Question Type: {result3['question_type']}")

New Session ID: 2c9edcb3-0e4e-40db-8108-c71284f56734
새 세션 응답:
Answer: 죄송하지만, 저는 당신의 이름을 알 수 없습니다. 개인정보를 보호하기 위해 사용자에 대한 정보를 저장하거나 기억하지 않습니다.
Question Type: FACT


## 5. 첫 번째 세션으로 돌아가서 대화 계속 (복원 확인)

In [6]:
# 원래 세션으로 돌아가서 대화 계속
result4 = await process_chat(
    message="좋습니다. 저는 어떤 일을 하고 있나요?",
    user_id=user_id,
    session_id=session_id  # 원래 세션 ID로 돌아감
)

print("원래 세션 복원 후 응답:")
print(f"Answer: {result4['answer']}")
print(f"Question Type: {result4['question_type']}")

원래 세션 복원 후 응답:
Answer: 죄송하지만, 저는 당신의 직업이나 일하는 분야에 대한 정보를 가지고 있지 않습니다. 당신이 어떤 일을 하고 있는지 말씀해 주시면 도움이 될 수 있도록 노력하겠습니다.
Question Type: FACT


In [8]:
from app.infra.checkpointer import get_postgres_checkpointer

# 저장된 체크포인트 확인
async with await get_postgres_checkpointer() as checkpointer:
    # 특정 세션의 체크포인트 목록 가져오기
    config = {"configurable": {"thread_id": session_id}}

    # 체크포인트 히스토리 조회
    history = []
    async for checkpoint in checkpointer.alist(config):
        history.append(checkpoint)

    print(f"Session {session_id}의 체크포인트 개수: {len(history)}")

    if history:
        print("\n최신 체크포인트:")
        latest = history[0]

        # CheckpointTuple 속성 확인
        print(f"Config: {latest.config}")

        # 체크포인트 데이터 확인
        checkpoint_data = latest.checkpoint
        print(f"Checkpoint keys: {list(checkpoint_data.keys())}")

        # 메시지 확인
        channel_values = checkpoint_data.get('channel_values', {})
        messages = channel_values.get('messages', [])
        print(f"💬 Messages: {len(messages)}개")

        # 메시지 내용 출력
        if messages:
            print("\n📋 저장된 메시지들:")
            for i, msg in enumerate(messages):
                if hasattr(msg, 'content'):
                    msg_type = 'user' if 'Human' in type(msg).__name__ else 'assistant'
                    print(f"  {i+1}. [{msg_type}]: {msg.content[:100]}...")
                else:
                    print(f"  {i+1}. [unknown]: {str(msg)[:100]}...")

Session c5da5b70-041e-4252-8f24-717c474e3c82의 체크포인트 개수: 12

최신 체크포인트:
Config: {'configurable': {'thread_id': 'c5da5b70-041e-4252-8f24-717c474e3c82', 'checkpoint_ns': '', 'checkpoint_id': '1f10c91e-0470-6c40-800a-efb40fadd3b2'}}
Checkpoint keys: ['v', 'id', 'ts', 'versions_seen', 'channel_values', 'channel_versions', 'updated_channels']
💬 Messages: 6개

📋 저장된 메시지들:
  1. [user]: 안녕하세요. 저는 김철수입니다....
  2. [assistant]: 안녕하세요, 김철수님! 만나서 반갑습니다. 어떻게 도와드릴까요?...
  3. [user]: 제 이름이 뭐라고 했죠?...
  4. [assistant]: 당신의 이름은 김철수라고 하셨습니다. 맞나요?...
  5. [user]: 좋습니다. 저는 어떤 일을 하고 있나요?...
  6. [assistant]: 죄송하지만, 저는 당신의 직업이나 일하는 분야에 대한 정보를 가지고 있지 않습니다. 당신이 어떤 일을 하고 있는지 말씀해 주시면 도움이 될 수 있도록 노력하겠습니다....
